# 探索差别隐私

在机器学习项目中，通常涉及通过迭代的数据分析过程来洞察数据，并确定哪些变量最有可能帮助构建预测模型。分析数据通常涉及到聚合和统计功能，通过这些功能可以了解变量的统计分布情况以及它们之间的关系。如果数据量很大，聚合功能可以为结果实现一定程度的抽象化；但是如果数据量较小，或者存在重复的分析，那么即使是聚合后的结果也可能会暴露个别观测值的详细信息。

差分隐私这种技术可通过对数据添加“干扰”来保护单个数据点隐私。其目标是确保添加足够的干扰以实现单个值的隐私性，同时确保数据的总体统计组成不变，并且聚合在统计上产生的结果与使用原始数据时相似。

## 安装 SmartNoise SDK

[*SmartNoise*](https://smartnoise.org/) 是来自 OpenDP 的工具包，这是一个 Microsoft 和哈佛大学的研究人员以及其他贡献者共同合作的项目，旨在为在数据分析和机器学习项目中使用差异隐私提供构建块。

> **备注**：SmartNoise 目前处于开发的早期阶段。

首先，我们要安装 SmartNoise Python SDK 包。可以忽略此练习中有关 Azure CLI 兼容性的任何错误。

In [ ]:
!pip install opendp-smartnoise==0.1.4.2

## 加载数据

现在让我们来看看一些原始数据。在本例中，我们有 10,000 个已接受糖尿病检测的患者记录。

In [ ]:
import pandas as pd

data_path = 'data/diabetes.csv'
diabetes = pd.read_csv(data_path)
diabetes.describe()

以上代码的输出显示了糖尿病数据集中变量的关键汇总统计信息。

## 执行分析

你可以使用 SmartNoise 创建一个能为源数据添加干扰的分析。干扰添加之下的数学运算可能相当复杂，不过 SmartNoise 能帮助你处理大部分细节问题。但你需要了解一些概念。

- **上限和下限**：钳位用于设置变量值的上限和下限。这是为了确保 SmartNoise 产生的干扰与原始数据的预期分布一致。
- **样本大小**：要为部分聚合生成一致的差分隐私数据，SmartNoise 需要知道要生成的数据样本的大小。
- **Epsilon**：简单地说，*epsilon* 是一个非负值，它提供了对添加到数据中的干扰量的反向度量。如果 epsilon 较低，数据集的隐私级别就更高；如果 epsilon 较高，则数据集会更接近原始数据。Epsilon 的取值通常在 0 到 1 之间。Epsilon 与另一个名为 *delta* 的值相关，该值表示的是某项分析生成的报告并不是完全隐私的概率。

记住这些概念后，检查并运行以下代码，此代码会创建一个分析并报告来自差分隐私数据的平均“**年龄**”值。还显示了初始数据的实际平均值以供比较。

In [ ]:
import opendp.smartnoise.core as sn

cols = list(diabetes.columns)
age_range = [0.0, 120.0]
samples = len(diabetes)

with sn.Analysis() as analysis:
    # load data
    data = sn.Dataset(path=data_path, column_names=cols)
    
    # Convert Age to float
    age_dt = sn.to_float(data['Age'])
    
    # get mean of age
    age_mean = sn.dp_mean(data = age_dt,
                          privacy_usage = {'epsilon': .50},
                          data_lower = age_range[0],
                          data_upper = age_range[1],
                          data_rows = samples
                         )
    
analysis.release()

# print differentially private estimate of mean age
print("Private mean age:",age_mean.value)

# print actual mean age
print("Actual mean age:",diabetes.Age.mean())

## 用直方图探索数据分布情况

直方图常用于分析数据以检查变量分布。

例如，让我们看看糖尿病数据集中年龄的真实分布情况。

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

ages = list(range(0, 130, 10))
age = diabetes.Age

# Plot a histogram with 10-year bins
n_age, bins, patches = plt.hist(age, bins=ages, color='blue', alpha=0.7, rwidth=0.85)
plt.grid(axis='y', alpha=0.75)
plt.xlabel('Age')
plt.ylabel('Frequency')
plt.title('True Age Distribution')
plt.show()
print(n_age.astype(int))

现在让我们来比照一下“年龄”的差分隐私直方图。

In [ ]:
import matplotlib.pyplot as plt

with sn.Analysis() as analysis:
    data = sn.Dataset(path = data_path, column_names = cols)

    age_histogram = sn.dp_histogram(
            sn.to_int(data['Age'], lower=0, upper=120),
            edges = ages,
            upper = 10000,
            null_value = -1,
            privacy_usage = {'epsilon': 0.5}
        )
    
analysis.release()

plt.ylim([0,7000])
width=4
agecat_left = [x + width for x in ages]
agecat_right = [x + 2*width for x in ages]
plt.bar(list(range(0,120,10)), n_age, width=width, color='blue', alpha=0.7, label='True')
plt.bar(agecat_left, age_histogram.value, width=width, color='orange', alpha=0.7, label='Private')
plt.legend()
plt.title('Histogram of Age')
plt.xlabel('Age')
plt.ylabel('Frequency')
plt.show()

print(age_histogram.value)

两个直方图的相似程度足以确保基于差分隐私数据的报告与源自原始数据的报告能提供一致的见解。

## 计算协方差

分析的另一个共同目标是建立变量之间的关系。SmartNoise 提供了一个差分隐私协方差函数，可以帮助解决这个问题。

In [ ]:
with sn.Analysis() as analysis:
    sn_data = sn.Dataset(path = data_path, column_names = cols)

    age_bp_cov_scalar = sn.dp_covariance(
                left = sn.to_float(sn_data['Age']),
                right = sn.to_float(sn_data['DiastolicBloodPressure']),
                privacy_usage = {'epsilon': 1.0},
                left_lower = 0.,
                left_upper = 120.,
                left_rows = 10000,
                right_lower = 0.,
                right_upper = 150.,
                right_rows = 10000)
analysis.release()
print('Differentially private covariance: {0}'.format(age_bp_cov_scalar.value[0][0]))
print('Actual covariance', diabetes.Age.cov(diabetes.DiastolicBloodPressure))

在本例中，**Age** 与 **DisatolicBloodPressure** 之间的协方差为正，说明年龄越大，血压越高。

## 使用 SQL 查询

除了**分析**功能之外，通过 SmartNoise 还能对数据源使用 SQL 查询以检索差分隐私聚合生成的结果。

首先，需要为数据架构中的表定义元数据。你可以在 .yml 文件中执行此操作，例如 **/metadata** 文件夹中的 **diabetes.yml** 文件。元数据描述表中的字段，包括数据类型以及数值字段的最小值和最大值。

In [ ]:
from opendp.smartnoise.metadata import CollectionMetadata

meta = CollectionMetadata.from_file('metadata/diabetes.yml')
print (meta)

定义了元数据后，可以创建你可以查询的读取器。在下面的示例中，我们将创建一个 **PandasReader** 来读取 Pandas 数据帧中的原始数据，并创建一个 **PrivateReader**，它将为 **PandasReader** 添加差分隐私层。

In [ ]:
from opendp.smartnoise.sql import PandasReader, PrivateReader

reader = PandasReader(diabetes, meta)
private_reader = PrivateReader(reader=reader, metadata=meta, epsilon_per_column=0.7)
print('Readers ready.')

现在，你可以向隐私读取器提交一个返回聚合结果集的 SQL 查询。

In [ ]:
query = 'SELECT Diabetic, AVG(Age) AS AvgAge FROM diabetes.diabetes GROUP BY Diabetic'

result_dp = private_reader.execute(query)
print(result_dp)

我们来比较该结果和源于原始数据的相同聚合。

In [ ]:
result = reader.execute(query)
print(result)

可以使用 **epsilon_per_column** 参数自定义 **PrivateReader** 的行为。

我们来尝试对一个读取器采用较高的 epsilon 值（即隐私度较低）并对另一个读取器采用较低的 epsilon 值（即隐私度较高）。

In [ ]:
low_privacy_reader = PrivateReader(reader, meta, 5.0)  # large epsilon, less privacy
result = low_privacy_reader.execute(query)
print(result)
print()

high_privacy_reader = PrivateReader(reader, meta, 0.1)  # smaller epsilon, more privacy
result = high_privacy_reader.execute(query)
print(result)

请注意，高 epsilon 值（隐私度较低）读取器的结果比低 epsilon 值（隐私度较高）读取器的结果更接近源于原始数据的真实结果。

## 拓展阅读

要详细了解 SmartNoise 提供的差分隐私功能，请参阅 [https://smartnoise.org](https://smartnoise.org/)